# <center>________________________________________________________________</center>

# <center>SQL QUERIES FOR SPACEX FALCON 9 LAUNCH DATA</center>

# <center>________________________________________________________________</center>

# Introduction
***

In this project, we will create some SQL queries to gather information from SpaceX Launch Data. The dataset includes a record for each payload carried with Falcon 9 rocket during a SpaceX mission into outer space.

# Libraries
***

In [ ]:
#!pip install pandas
#!pip install sqlalchemy
#!pip install sqlite3
#!pip install ipython-sql

In [ ]:
import csv, sqlite3
import pandas as pd

# Data Acquisition and Database Connection
***

First we will load the SQL extension, import our data into a database, and establish a connection with the database:

In [ ]:
%load_ext sql

In [ ]:
con = sqlite3.connect("my_database.db")
cur = con.cursor()

In [ ]:
%sql sqlite:///my_database.db

In [ ]:
df = pd.read_csv("https://github.com/efeyemez/Portfolio/raw/main/Datasets/SpaceX_Falcon_9/launch_data.csv")
print(df.shape)
df.head()

In [ ]:
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

# Queries
***

### Names of the unique launch sites  in the space mission:

In [ ]:
%%sql

SELECT DISTINCT "Launch_Site" FROM SPACEXTBL;

### 5 records where launch sites begin with the string 'CCA':

In [ ]:
%%sql

SELECT * FROM SPACEXTBL
    WHERE "Launch_Site" LIKE "CCA%" LIMIT 5;

### Total payload mass carried by boosters launched by NASA (CRS):

In [ ]:
%%sql

SELECT SUM("PAYLOAD_MASS__KG_") AS "Total Payload Mass (kg) by NASA (CRS)" FROM SPACEXTBL
    WHERE "Customer" = "NASA (CRS)";

### Average payload mass carried by booster version F9 v1.1:

In [ ]:
%%sql

SELECT AVG("PAYLOAD_MASS__KG_") AS "Average Payload Mass (kg) by F9 v1.1" FROM SPACEXTBL
    WHERE "Booster_Version" = "F9 v1.1";

### Date when the first succesful landing on a ground pad was acheived:

In [ ]:
%%sql

SELECT MIN(substr("Date",7,4) || substr("Date",4,2) || substr("Date",1,2)) as "Date (YYYYMMDD)" FROM SPACEXTBL
    WHERE "Landing_Outcome" = "Success (ground pad)";

### Versions of the boosters that landed successfully on a drone ship with a payload between 4000 and 6000 kg:

In [ ]:
%%sql

SELECT DISTINCT "Booster_Version" FROM SPACEXTBL
    WHERE "Landing_Outcome" = "Success (drone ship)" AND ("PAYLOAD_MASS__KG_" > 4000 AND "PAYLOAD_MASS__KG_" < 6000);

### Total number of successful and failed missions:

In [ ]:
%%sql

SELECT DISTINCT "Mission_Outcome" AS "Mission Outcome", COUNT("Mission_Outcome") AS "Count" FROM SPACEXTBL
    GROUP BY "Mission_Outcome";

### Versions of the boosters that carried the maximum payload (with a subquery):

In [ ]:
%%sql

SELECT "Booster_Version" FROM SPACEXTBL
    WHERE "PAYLOAD_MASS__KG_" = (SELECT MAX("PAYLOAD_MASS__KG_") FROM SPACEXTBL);

### Failed drone ship landings in 2015, with respective booster versions, launch sites, and the month of the year:

In [ ]:
%%sql

SELECT substr("Date", 4, 2) AS "Month of the Year 2015", "Landing_Outcome", "Booster_Version", "Launch_Site" FROM SPACEXTBL
    WHERE "Landing_Outcome" = "Failure (drone ship)" AND substr(Date,7,4)='2015';

### Successful landings between 4th of June, 2010 and 20th of March, 2017 in descending order:

In [ ]:
%%sql

SELECT "Date", "Landing_Outcome" FROM SPACEXTBL
    WHERE substr("Landing_Outcome", 1, 7)="Success" AND
    (substr("Date",7,4) || substr("Date",4,2) || substr("Date",1,2)) BETWEEN "20100604" AND "20170320"
    ORDER BY (substr("Date",7,4) || substr("Date",4,2) || substr("Date",1,2));

# <center>________________________________________________________________</center>